In [ ]:
import os
import sys
import math
import argparse
import numpy as np
from collections import Counter

# Keras imports
from tensorflow.keras.models import Model
from keras import backend as K

# ml4h Imports
from ml4h.arguments import parse_args
from ml4h.models import make_multimodal_multitask_model, train_model_from_generators
from ml4h.tensor_generators import TensorGenerator, big_batch_from_minibatch_generator, test_train_valid_tensor_generators

# IPython imports
from IPython.display import Image
%matplotlib inline
import matplotlib.pyplot as plt

# IPython imports
from IPython.display import Image

DATA_DIRECTORY = '/mnt/disks/ecg-rest-36k-tensors/2019-10-22/'
MODEL_DIRECTORY = '/mnt/ml4cvd/projects/jamesp/data/models/'

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
# ~~~~~ Scale with Temperature ~~~~~~
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
def temperature_scale(predictions, temperature=1.0):
	predictions = np.log(predictions) / temperature
	return 1 / (1 + np.exp(-predictions / temperature))

In [ ]:
sys.argv = ['train', 
            '--tensors', DATA_DIRECTORY, 
            '--input_tensors', 'ecg_rest', 
            '--output_tensors', 'ecg_rhythm',
            '--batch_size', '32', 
            '--epochs', '1',  
            '--learning_rate', '0.0001',
            '--model_file', MODEL_DIRECTORY + 'ecg_rest_rhythm_w_poor.hd5',
            '--test_steps', '96',
            '--inspect_model',
            '--id', 'ecg_rest_rhythm_w_poor']
args = parse_args()
generate_train, generate_valid, generate_test = test_train_valid_tensor_generators(args.tensor_maps_in,  args.tensor_maps_out,  args.tensors, args.batch_size,   args.valid_ratio, args.test_ratio, args.test_modulo, args.balance_csvs)
model = make_multimodal_multitask_model(**args.__dict__)
test_data, test_labels, test_paths = big_batch_from_minibatch_generator(args.tensor_maps_in, args.tensor_maps_out,
                                                                        generate_test, args.test_steps)

In [ ]:
predictions = model.predict(test_data)

In [ ]:
print(predictions.shape)

In [ ]:
def plot_predicted_distributions(predictions, n_bins=8, sharey=True, labels=None):
    sides = int(np.ceil(np.sqrt(predictions.shape[-1])))
    _, axes = plt.subplots(sides-1, sides, figsize=(12, 8), sharey=sharey)
    for i in range(predictions.shape[-1]):
        axes[i//sides, i%sides].hist(predictions[..., i], bins=n_bins)
        if labels is not None:
            axes[i//sides, i%sides].set_title(labels[i])

In [ ]:
print(args.tensor_maps_out[0].channel_map)
print('prediction:',np.sum(predictions, axis=0))
print('truth:', np.sum(test_labels['output_ecg_rhythm_categorical'], axis=0))

In [ ]:
labels = {v: k for k, v in args.tensor_maps_out[0].channel_map.items()}
plot_predicted_distributions(predictions, labels=labels, sharey=False)

In [ ]:
cold_p = temperature_scale(predictions, temperature=1.0)
plot_predicted_distributions(cold_p, labels=labels, sharey=True)

In [ ]:
cold_p = temperature_scale(predictions, temperature=0.1)
plot_predicted_distributions(cold_p, labels=labels, sharey=True)


In [ ]:
cold_p = temperature_scale(predictions, temperature=0.0001)
plot_predicted_distributions(cold_p, labels=labels, sharey=True)

In [ ]:
hot_p = temperature_scale(predictions, temperature=10.1)
plot_predicted_distributions(hot_p, labels=labels, sharey=False)

In [ ]:
hot_p = temperature_scale(predictions, temperature=15)
plot_predicted_distributions(hot_p, labels=labels, sharey=False, n_bins=100)

In [ ]:
plot_predicted_distributions(test_labels['output_ecg_rhythm_categorical'], labels=labels, sharey=False)


In [ ]:
Image('./recipes_output/ecg_rhythm/architecture_graph_ecg_rhythm.png')

In [ ]:
Image('./recipes_output/ecg_regresser/per_class_roc_ecg_rhythm.png')

In [ ]:
Image('./recipes_output/ecg_regresser/per_class_roc_ecg_normal.png')

In [ ]:
Image('./recipes_output/ecg_regresser/scatter_PPInterval.png')

In [ ]:
Image('./recipes_output/ecg_regresser/scatter_QRSDuration.png')